# Pre-Processing: ResNet (224x224)

Dieses Skript automatisiert die Bildvorbereitung.

**Deine Ordner-Struktur:**
* **Eingang:** `data/all/cup` (Hier liegen deine riesigen 4K Bilder)
* **Ausgang:** `data/cup` (Hier landen die fertigen 224x224 Bilder)

**Was passiert hier?**
1.  Das Skript schaut in `data/all` nach, welche Klassen (Ordner) es gibt.
2.  Es erstellt die exakt gleichen Ordner direkt unter `data/`.
3.  Es nimmt die riesigen Bilder, verkleinert sie drastisch auf 224x224 Pixel (ResNet50 Standard) und speichert sie im neuen Ordner.

In [2]:
import os
from PIL import Image

from pillow_heif import register_heif_opener

# 1. HEIC-Support aktivieren
register_heif_opener()

# --- KONFIGURATION ---
# Pfad zu den Originalen
SOURCE_DIR = "data/all"

# Pfad wo die fertigen Bilder landen sollen
TARGET_DIR = "data"

# Zielgröße für ResNet50
IMG_SIZE = (224, 224)

print("Konfiguration geladen.")
print(f"Quelle: {SOURCE_DIR}")
print(f"Ziel:   {TARGET_DIR}")

Konfiguration geladen.
Quelle: data/all
Ziel:   data


In [3]:
def process_images_with_heic():
    if not os.path.exists(SOURCE_DIR):
        print(f"Fehler: Ordner '{SOURCE_DIR}' nicht gefunden.")
        return

    all_items = os.listdir(SOURCE_DIR)
    count = 0

    print("Starte Verarbeitung (inkl. HEIC Support)...\n")

    for class_name in all_items:
        full_source_path = os.path.join(SOURCE_DIR, class_name)

        if os.path.isdir(full_source_path):
            full_target_path = os.path.join(TARGET_DIR, class_name)
            os.makedirs(full_target_path, exist_ok=True)

            print(f"-> Bearbeite Klasse: {class_name}")

            files = os.listdir(full_source_path)

            for filename in files:
                # Wir prüfen jetzt auch auf .heic und .HEIC
                if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp', '.heic')):
                    try:
                        src_file = os.path.join(full_source_path, filename)

                        # Ziel-Dateiname: Endung immer auf .jpg ändern!
                        # Wir entfernen die alte Endung und hängen .jpg dran
                        name_without_ext = os.path.splitext(filename)[0]
                        dst_filename = name_without_ext + ".jpg"
                        dst_file = os.path.join(full_target_path, dst_filename)

                        # Bild öffnen (PIL erkennt HEIC jetzt automatisch durch register_heif_opener)
                        img = Image.open(src_file)
                        img = img.convert('RGB')

                        # Resize
                        img_resized = img.resize(IMG_SIZE, Image.Resampling.LANCZOS)

                        # Speichern als JPG
                        img_resized.save(dst_file, quality=95)

                        count += 1

                    except Exception as e:
                        print(f"   Fehler bei Datei {filename}: {e}")

    print(f"\nFERTIG. {count} Bilder (inkl. HEIC) wurden verarbeitet.")

# Ausführen
process_images_with_heic()

Starte Verarbeitung (inkl. HEIC Support)...

-> Bearbeite Klasse: cup
-> Bearbeite Klasse: pen
-> Bearbeite Klasse: bottle
-> Bearbeite Klasse: keyboard

FERTIG. 100 Bilder (inkl. HEIC) wurden verarbeitet.
